## Test the environment `HerdEnv` of `herd_env.py`

In [1]:
import numpy as np
import tensorflow as tf
from tf_agents.environments import utils
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories import TimeStep
from tf_agents.policies import scripted_py_policy
from tf_agents.policies import random_py_policy
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver
from tf_agents.utils import common
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential

In [2]:
from RNN_Env import FREnv

In [3]:
# sanity check
henv_val = FREnv(herd_sizes = [64,64], rand_recovery_prob = 0.1, rand_infection_prob = 0.05)
utils.validate_py_environment(henv_val, episodes=10)

In [4]:
# create Herd Environment instance to be trained for
max_episode_length=1000
num_herds=2
henv = FREnv(herd_sizes = [32,32], expected_episode_length=100, max_episode_length=max_episode_length,
               rand_recovery_prob = 0.02, rand_infection_prob = 0.06)

In [5]:
# show interor values of environment
time_step = henv.reset()
print(time_step)
cumulative_reward = time_step.reward
finished = False

while not finished:
  time_step = henv.step([0.,0.]) # do nothing
  s = henv.get_state()
  print("state: ", s, "observation: ", time_step.observation, "\treward: ", time_step.reward)
  cumulative_reward += time_step.reward
  if time_step.step_type == StepType.LAST:
    finished = True

print('Final Reward = ', cumulative_reward)

TimeStep(
{'discount': 1.0,
 'observation': array([0., 0., 0.], dtype=float32),
 'reward': 0.0,
 'step_type': array(0, dtype=int32)})
state:  [2 0 2 0 1 1] observation:  [0.001 0.001 0.001] 	reward:  -0.05
state:  [3 2 5 2 2 2] observation:  [0.002 0.002 0.002] 	reward:  -0.175
state:  [3 3 8 5 3 3] observation:  [0.003 0.003 0.003] 	reward:  -0.325
state:  [ 5  6 13 11  4  4] observation:  [0.004 0.004 0.004] 	reward:  -0.6
state:  [ 9  8 22 19  5  5] observation:  [0.005 0.005 0.005] 	reward:  -1.025
state:  [ 9  8 31 27  6  6] observation:  [0.006 0.006 0.006] 	reward:  -1.45
state:  [12  7 43 34  7  7] observation:  [0.007 0.007 0.007] 	reward:  -1.925
state:  [13  8 56 42  8  8] observation:  [0.008 0.008 0.008] 	reward:  -2.45
state:  [14  9 70 51  9  9] observation:  [0.009 0.009 0.009] 	reward:  -3.025
state:  [16  9 86 60 10 10] observation:  [0.01 0.01 0.01] 	reward:  -3.65
state:  [ 17  10 103  70  11  11] observation:  [0.011 0.011 0.011] 	reward:  -4.325
state:  [ 18  10 1

In [6]:
action_spec = henv.action_spec()
ts_spec = henv.time_step_spec()
print("action spec:\n", action_spec, "\n\ntime step spec:\n", ts_spec)

action spec:
 BoundedArraySpec(shape=(2,), dtype=dtype('float32'), name='action', minimum=0.0, maximum=1.0) 

time step spec:
 TimeStep(
{'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': BoundedArraySpec(shape=(3,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0),
 'reward': ArraySpec(shape=(), dtype=dtype('float32'), name='reward'),
 'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')})


### Define scripted policies

In [14]:
# do nothing policy: cull never
action_script0 = [(max_episode_length, [0,0])]

# cull first herd every 20th step and second herd every 20th step
action_script1 = [(9, [0,0]), 
                 (1, [1,1]),
                 (9, [0,0]), 
                 (1, [1,1])] * int(1+max_episode_length/20)

manual_scripted_policy0 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=ts_spec,
    action_spec=action_spec,
    action_script=action_script0)

manual_scripted_policy1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=ts_spec,
    action_spec=action_spec,
    action_script=action_script1)

init_policy_state = manual_scripted_policy0.get_initial_state()

In [15]:
policy_state =  init_policy_state
ts0 = henv.reset()
for _ in range(21):
    action_step = manual_scripted_policy1.action(ts0, policy_state)
    policy_state = action_step.state
    print("action=", action_step.action, "\tpolicy_state", policy_state)
policy_state = manual_scripted_policy1.get_initial_state()

action= [0. 0.] 	policy_state [0, 1]
action= [0. 0.] 	policy_state [0, 2]
action= [0. 0.] 	policy_state [0, 3]
action= [0. 0.] 	policy_state [0, 4]
action= [0. 0.] 	policy_state [0, 5]
action= [0. 0.] 	policy_state [0, 6]
action= [0. 0.] 	policy_state [0, 7]
action= [0. 0.] 	policy_state [0, 8]
action= [0. 0.] 	policy_state [0, 9]
action= [1. 1.] 	policy_state [1, 1]
action= [0. 0.] 	policy_state [2, 1]
action= [0. 0.] 	policy_state [2, 2]
action= [0. 0.] 	policy_state [2, 3]
action= [0. 0.] 	policy_state [2, 4]
action= [0. 0.] 	policy_state [2, 5]
action= [0. 0.] 	policy_state [2, 6]
action= [0. 0.] 	policy_state [2, 7]
action= [0. 0.] 	policy_state [2, 8]
action= [0. 0.] 	policy_state [2, 9]
action= [1. 1.] 	policy_state [3, 1]
action= [0. 0.] 	policy_state [4, 1]


### ... and a random policy

In [16]:
random_policy = random_py_policy.RandomPyPolicy(time_step_spec=ts_spec, action_spec=action_spec)

## Drive a rollout

In [17]:
def compute_avg_return(environment, policy, num_episodes=50, verbose=False):
  total_return = 0.0
  cullsteps = 0
  for e in range(num_episodes):

    time_step = environment.reset()
    if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
        policy_state = policy.get_initial_state() # remember where in the script we were
    else:
        policy_state = None # other policies without memory
    episode_return = 0.0
    i=0
    while not time_step.is_last():
        i+=1
        action_step = policy.action(time_step, policy_state)
        if action_step.action[0] > 0 or action_step.action[1] > 0:
            cullsteps += 1
        policy_state = action_step.state
        time_step = environment.step(action_step.action)
        if isinstance(environment, FREnv):
            state = environment.get_state()
        else:
            state = None # TF environment from wrapper does not have get_state()
        episode_return += time_step.reward
        if verbose:
            print (f"episode {e:>2} step{i:>4} action: ", action_step.action, "state=", state, "obs=", time_step.observation, "reward=", time_step.reward)
    total_return += episode_return

  avg_return = total_return / num_episodes
  cullsteps /= num_episodes
  return avg_return, cullsteps

In [18]:
random_reward, cullsteps = compute_avg_return(henv, random_policy)
print (f"average return of random policy: {random_reward:.3f} avg steps with culls per episode: {cullsteps}")

average return of random policy: -2986.306 avg steps with culls per episode: 75.8


In [19]:
# show states for one rollout of second scripted policy
compute_avg_return(henv, manual_scripted_policy1, num_episodes=1, verbose=True)

episode  0 step   1 action:  [0. 0.] state= [2 1 2 1 1 1] obs= [0.001 0.001 0.001] reward= -0.075
episode  0 step   2 action:  [0. 0.] state= [2 3 4 4 2 2] obs= [0.002 0.002 0.002] reward= -0.2
episode  0 step   3 action:  [0. 0.] state= [ 6  6 10 10  3  3] obs= [0.003 0.003 0.003] reward= -0.5
episode  0 step   4 action:  [0. 0.] state= [ 8  6 18 16  4  4] obs= [0.004 0.004 0.004] reward= -0.85
episode  0 step   5 action:  [0. 0.] state= [10  9 28 25  5  5] obs= [0.005 0.005 0.005] reward= -1.325
episode  0 step   6 action:  [0. 0.] state= [10 10 38 35  6  6] obs= [0.006 0.006 0.006] reward= -1.825
episode  0 step   7 action:  [0. 0.] state= [12 14 50 49  7  7] obs= [0.007 0.007 0.007] reward= -2.475
episode  0 step   8 action:  [0. 0.] state= [12 15 62 64  8  8] obs= [0.008 0.008 0.008] reward= -3.15
episode  0 step   9 action:  [0. 0.] state= [15 16 77 80  9  9] obs= [0.009 0.009 0.009] reward= -3.925
episode  0 step  10 action:  [1. 1.] state= [ 0  0 77 80  0  0] obs= [0.01 0.   0.

(-568.325000077486, 4.0)

In [20]:
manual_reward0, cullsteps = compute_avg_return(henv, manual_scripted_policy0, num_episodes=200)
print (f"average return of do-nothing-policy: {manual_reward0:.3f} avg culls {cullsteps}")
manual_reward1, cullsteps = compute_avg_return(henv, manual_scripted_policy1, num_episodes=200)
print (f"average return of manual policy: {manual_reward1:.3f} avg culls {cullsteps}")

average return of do-nothing-policy: -16469.791 avg culls 0.0
average return of manual policy: -4017.298 avg culls 9.365


### Train a DDPG Agent

In [144]:
def compute_avg_return(environment, policy, num_episodes=50, verbose=False):
  total_return = 0.0
  cullsteps = 0
  for e in range(num_episodes):

    time_step = environment.reset()
    if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
        policy_state = policy.get_initial_state() # remember where in the script we were
    else:
        #print(policy.get_initial_state(batch_size=train_env.batch_size()))
        policy_state = policy.get_initial_state(batch_size=1) # other policies without memory
    episode_return = 0.0
    i=0
    while not time_step.is_last():
        i+=1
        action_step = policy.action(time_step, policy_state)
        if action_step.action[0][0] > 0 or action_step.action[0][1] > 0:
            cullsteps += 1
        policy_state = action_step.state
        time_step = environment.step(action_step.action)
        if isinstance(environment, FREnv):
            state = environment.get_state()
        else:
            state = None # TF environment from wrapper does not have get_state()
        episode_return += time_step.reward
        if verbose:
            print (f"episode {e:>2} step{i:>4} action: ", action_step.action, "state=", state, "obs=", time_step.observation, "reward=", time_step.reward)
    total_return += episode_return

  avg_return = total_return / num_episodes
  cullsteps /= num_episodes
  return avg_return, cullsteps

In [145]:
'''Made it work to here, implement ddpg agent next.'''

import functools
from tf_agents.networks import nest_map
from tf_agents.networks import sequential
from tf_agents.keras_layers import inner_reshape
from tf_agents.networks.sequential import Sequential
from tensorflow.keras.layers import Dense
from tf_agents.agents.ddpg import ddpg_agent
from tf_agents.agents.ddpg import actor_rnn_network
from tf_agents.agents.ddpg import critic_rnn_network
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory

In [146]:
num_iterations = 30000
replay_buffer_max_length = 10000
batch_size = 64
num_eval_episodes = 100
initial_collect_steps = 1000
collect_steps_per_iteration = 5
log_interval = 500
eval_interval = 500
target_update_period = 5

In [147]:
train_env = tf_py_environment.TFPyEnvironment(FREnv(herd_sizes = [32,32], expected_episode_length=100, max_episode_length=max_episode_length,
               rand_recovery_prob = 0.02, rand_infection_prob = 0.06))
eval_env = tf_py_environment.TFPyEnvironment(FREnv(herd_sizes = [32,32], expected_episode_length=100, max_episode_length=max_episode_length,
               rand_recovery_prob = 0.02, rand_infection_prob = 0.06))

In [177]:
#kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.03, maxval=0.03)

#Make critic and actor net for ddpg

actor_fc_layers=(400, 300)
actor_output_fc_layers=(100,)
actor_lstm_size=(40,)
critic_obs_fc_layers=(400,)
critic_action_fc_layers=None
critic_joint_fc_layers=(300,)
critic_output_fc_layers=(100,)
critic_lstm_size=(40,)


actor_net = actor_rnn_network.ActorRnnNetwork(
        train_env.time_step_spec().observation,
        train_env.action_spec(),
        input_fc_layer_params=actor_fc_layers,
        lstm_size=actor_lstm_size,
        output_fc_layer_params=actor_output_fc_layers)

critic_net_input_specs = (train_env.time_step_spec().observation,
                              train_env.action_spec())

critic_net = critic_rnn_network.CriticRnnNetwork(
        critic_net_input_specs,
        observation_fc_layer_params=critic_obs_fc_layers,
        action_fc_layer_params=critic_action_fc_layers,
        joint_fc_layer_params=critic_joint_fc_layers,
        lstm_size=critic_lstm_size,
        output_fc_layer_params=critic_output_fc_layers)

In [190]:
train_step_counter = tf.Variable(0)
global_step = tf.compat.v1.train.get_or_create_global_step()

agent = ddpg_agent.DdpgAgent(
        train_env.time_step_spec(),
        train_env.action_spec(),
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=1e-4),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=1e-3),
        ou_stddev=0.2,
        ou_damping=0.15,
        target_update_tau=0.05,
        target_update_period=5,
        #dqda_clipping=dqda_clipping,
        td_errors_loss_fn=tf.compat.v1.losses.huber_loss,
        gamma=0.99,
        #reward_scale_factor=reward_scale_factor,
        #gradient_clipping=gradient_clipping,
        #debug_summaries=debug_summaries,
        #summarize_grads_and_vars=summarize_grads_and_vars,
        train_step_counter=global_step)

agent.initialize()

In [191]:
# agent.policy.trainable_variables

In [192]:
# manually initialize a reasonably good policy: kill both herds if the sum of observations is large
#W = np.array([[0, 3 ,0, 2],[0, 0, 3, 2,]])
#b = np.array([1, 0, 0, 0])
#q_net.layers[0].set_weights([W,b])
#agent.policy.trainable_variables

In [193]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

In [194]:
agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

In [195]:
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

    # Add trajectory to the replay buffer
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
    for _ in range(steps):
        collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

In [196]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

iterator = iter(dataset)
dataset

<PrefetchDataset shapes: (Trajectory(
{action: (64, 2, 2),
 discount: (64, 2),
 next_step_type: (64, 2),
 observation: (64, 2, 3),
 policy_info: (),
 reward: (64, 2),
 step_type: (64, 2)}), BufferInfo(ids=(64, 2), probabilities=(64,))), types: (Trajectory(
{action: tf.float32,
 discount: tf.float32,
 next_step_type: tf.int32,
 observation: tf.float32,
 policy_info: (),
 reward: tf.float32,
 step_type: tf.int32}), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [197]:
# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  tra = agent._as_transition(experience)
  ts, b, c = tra
  print(ts)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step <= 1 or step % log_interval == 0:
    print('step = {0:4>}: loss = {1:.4f}'.format(step, train_loss), end="\t")

  if step <= 1 or (step <= 100 and step % 50 == 0) or step % eval_interval == 0:
    avg_return, cullsteps = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step {0}: average return = {1:.1f} cullsteps = {2:.1f}'.format(step, avg_return.numpy().item(), cullsteps))
    returns.append(avg_return)

TimeStep(
{'discount': <tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>,
 'observation': <tf.Tensor: shape=(64, 1, 3), dtype=float32, numpy=
array([[[0

InvalidArgumentError: Invalid reduction dimension (1 for input with 0 dimension(s) [Op:Sum]

In [ ]:
learned_reward, culleps = compute_avg_return(eval_env, agent.policy, num_episodes=500)
print ("reward of learned policy: ", learned_reward.numpy(), "cullsteps=", culleps)

In [ ]:
init_ts = eval_env.reset()

def get_action(obs):
    """ execute the learned policy network 
       obs:  one float for global time, one float for each herd - the time since last culling
    """
    _ts = TimeStep(tf.constant([0.]),
                   tf.constant([0.]),
                   tf.constant([1]),
                   tf.constant([obs]))
    # a = agent.collect_policy.action(_ts) # just to see how much is explored versus exploited
    a = agent.policy.action(_ts)
    return a.action.numpy().item()

In [ ]:
agent.policy

In [ ]:
# what the learned policy does on a grid of observations (5 steps per row&col)
'''A = [[get_action([.0, x,y])
 for y in np.arange(0.,1.,.05,np.float32)]
 for x in np.arange(0.,1.,.05,np.float32)]
A'''

### Play with parameters of manually designed q_network policy

In [ ]:
# W, b = agent.policy.trainable_variables
# W = W.numpy()
# b = b.numpy()
# print ("weights\n", W, "\nbias", b)

In [ ]:
# def nn(obs):
#    y = np.dot(obs, W)+b
#    return y

In [ ]:
# nn([0.5,.2])